# 輸送最適化問題の解決

In [1]:
#!pip3 install pulp
#!pip3 install ortoolpy

In [2]:
import numpy as np
import pandas as pd
from itertools import product
from pulp import LpVariable, lpSum, value
from ortoolpy import model_min, addvars, addvals

df_tc = pd.read_csv("trans_cost.csv", index_col="工場")
df_demand = pd.read_csv("demand.csv")
df_supply = pd.read_csv("supply.csv")

In [3]:
df_tc.head()

,F1,F2,F3,F4
工場,,,,
W1,10,10,11,27
W2,18,21,12,14
W3,15,12,14,12


In [4]:
df_demand.head()

,F1,F2,F3,F4
0,28,29,31,25


In [5]:
df_supply.head()

,W1,W2,W3
0,35,41,42


In [6]:
#初期設定
np.random.seed(1)
nw = len(df_tc.index)
nf = len(df_tc.columns)
pr = list(product(range(nw), range(nf)))
print(nw)
print(nf)
print(pr)

3
4
[(0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3), (2, 0), (2, 1), (2, 2), (2, 3)]


In [7]:
# 数理モデル作成 最小化するモデル
m1 = model_min()

# 解決する変数の定義 (3行 * 4列の座標位置)
v1 = {(i,j):LpVariable("v%d_%d"%(i,j),lowBound=0) for i, j in pr}
#print(v1)
# 目的関数の定義 輸送ルートコストと(変数:倉庫から工場への配送量)の積の和(総コスト)が最小となるモデル。
# solveでこの変数が解決される
m1 += lpSum(df_tc.iloc[i][j]*v1[i,j] for i,j in pr)
# 制約条件
for i in range(nw):
    m1 += lpSum(v1[i,j] for j in range(nf)) <= df_supply.iloc[0][i]
for j in range(nf):
    m1 += lpSum(v1[i,j] for i in range(nw)) >= df_demand.iloc[0][j]
m1.solve()

df_tr_sol = df_tc.copy()
total_cost = 0
for k,x in v1.items():
    i,j = k[0],k[1]
    df_tr_sol.iloc[i][j] = value(x)
    total_cost += df_tc.iloc[i][j]*value(x)

print(df_tr_sol)
print("総輸送コスト:"+str(total_cost))

    F1  F2  F3  F4
工場                
W1  28   7   0   0
W2   0   0  31   5
W3   0  22   0  20
総輸送コスト:1296.0


# 62 最適輸送ルートをネットワークで確認する

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

df_tr = df_tr_sol.copy()
df_pos = pd.read_csv("trans_route_pos.csv")

G = nx.Graph()

for i in range(len(df_pos.columns)):
    G.add_node(df_pos.columns[i])

num_pre = 0
edge_weights = []
size = 0.1
for i in range(len(df_pos.columns)):
    for j in range(len(df_pos.columns)):
        if not (i==j):
            G.add_edge(df_pos.columns[i], df_pos.columns[j])
            if num_pre < len(G.edges):
                num_pre = len(G.edges)
                weight = 0
                if (df_pos.columns[i] in df_tr.columns)and(df_pos.columns[j] in df_tr.index):
                    if df_tr[df_pos.columns[i]][df_pos.columns[j]]:
                        weight = df_tr[df_pos.columns[i]][df_pos.columns[j]]*size
                elif (df_pos.columns[j] in df_tr.columns)and(df_pos.columns[i] in df_tr.index):
                    if df_tr[df_pos.columns[j]][df_pos.columns[i]]:
                        weight = df_tr[df_pos.columns[j]][df_pos.columns[i]]*size
                edge_weights.append(weight)
                
pos = {}
for i in range(len(df_pos.columns)):
    node = df_pos.columns[i]
    pos[node] = (df_pos[node][0], df_pos[node][1])

nx.draw(G, pos, with_labels=True, font_size=16, node_size=1000,
       node_color="k", font_color="w", width=edge_weights)

plt.show()

<Figure size 640x480 with 1 Axes>

# 63 最適ルートが制約条件を満たしているか確認する

In [9]:
import pandas as pd
import numpy as np

df_demand = pd.read_csv("demand.csv")
df_supply = pd.read_csv("supply.csv")

def condition_demand(df_tr, df_demand):
    flag = np.zeros(len(df_demand.columns))
    for i in range(len(df_demand.columns)):
        temp_sum = sum(df_tr[df_demand.columns[i]])
        if (temp_sum >= df_demand.iloc[0][i]):
            flag[i] = 1
    return flag

def condition_supply(df_tr, df_supply):
    flag = np.zeros(len(df_supply.columns))
    for i in range(len(df_supply.columns)):
        temp_sum = sum(df_tr.loc[df_supply.columns[i]])
        if temp_sum <= df_supply.iloc[0][i]:
            flag[i] = 1
    return flag

print("需要条件計算結果:"+str(condition_demand(df_tr_sol, df_demand)))
print("供給条件計算結果:"+str(condition_supply(df_tr_sol, df_supply)))

需要条件計算結果:[1. 1. 1. 1.]
供給条件計算結果:[1. 1. 1.]


# 64 生産計画に関するデータの読み込み

In [10]:
import pandas as pd

#製造に必要な原料の割合
df_material = pd.read_csv("product_plan_material.csv", index_col="製品")
print(df_material)
#製品の利益
df_profit = pd.read_csv("product_plan_profit.csv", index_col="製品")
print(df_profit)
#原料の在庫
df_stock = pd.read_csv("product_plan_stock.csv", index_col="項目")
print(df_stock)
#製品の生産量
df_plan = pd.read_csv("product_plan.csv", index_col="製品")
print(df_plan)

     原料1  原料2  原料3
製品                
製品1    1    4    3
製品2    2    4    1
      利益
製品      
製品1  5.0
製品2  4.0
    原料1  原料2  原料3
項目               
在庫   40   80   50
      生産量
製品       
製品1    16
製品2     0


# 65 利益を計算する関数の作成

In [11]:
def product_plan(df_profit, df_plan):
    profit = 0
    for i in range(len(df_profit.index)):
        for j in range(len(df_plan.columns)):
            profit += df_profit.iloc[i][j] * df_plan.iloc[i][j]
    return profit

print("総利益:"+str(product_plan(df_profit, df_plan)))

総利益:80.0


# 66 生産最適化問題の解決

In [12]:
import pandas as pd
from pulp import LpVariable, lpSum, value
from ortoolpy import model_max, addvars, addvals

df = df_material.copy()
inv = df_stock

m = model_max()
# 答え:製品に対しての生産量を格納するdict変数
v1 = {(i):LpVariable("v%d"%(i),lowBound=0) for i in range(len(df_profit))}
m += lpSum(df_profit.iloc[i]*v1[i] for i in range(len(df_profit)))
for i in range(len(df_material.columns)):
    # 各原料が在庫数を超えない制約を作成
    m += lpSum(df_material.iloc[j,i]*v1[j] for j in range(len(df_profit))) <= df_stock.iloc[:,i]
m.solve()

df_plan_sol = df_plan.copy()
for k,x in v1.items():
    df_plan_sol.iloc[k] = value(x)
print(df_plan_sol)
print("総利益:"+str(value(m.objective)))


      生産量
製品       
製品1  15.0
製品2   5.0
総利益:95.0


# 67 最適化生産計画が条件内であるか確認

In [13]:
def condition_stock(df_plan, df_material, df_stock):
    flag = np.zeros(len(df_material.columns))
    for i in range(len(df_material.columns)):
        temp_sum = 0
        for j in range(len(df_material.index)):
            temp_sum = temp_sum + df_material.iloc[j][i]*float(df_plan.iloc[j])
        if (temp_sum <= float(df_stock.iloc[0][i])):
            flag[i] = 1
        print(df_material.columns[i]+"使用量:"+str(temp_sum)+" 在庫:"
              +str(float(df_stock.iloc[0][i])))
    return flag

print("制約条件計算結果"+str(condition_stock(df_plan_sol, df_material, df_stock)))

原料1使用量:25.0 在庫:40.0
原料2使用量:80.0 在庫:80.0
原料3使用量:50.0 在庫:50.0
制約条件計算結果[1. 1. 1.]
